<a href="https://colab.research.google.com/github/aghapygad336/MENA/blob/master/MENA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [148]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [171]:
import pandas as pd
import numpy as np
data2015 = pd.read_csv('/content/drive/My Drive/MENA/2015.csv',encoding = "ISO-8859-1", engine = 'python')
dataLabels2015=np.array(data2015.columns)
data2016 = pd.read_csv('/content/drive/My Drive/MENA/2016.csv',encoding = "ISO-8859-1", engine = 'python')
dataLabels2016=np.array(data2016.columns)
data2017 = pd.read_csv('/content/drive/My Drive/MENA/2017.csv',encoding = "ISO-8859-1", engine = 'python')

data = pd.read_csv('/content/drive/My Drive/MENA/golbal.csv',encoding = "ISO-8859-1", engine = 'python', usecols=[1,2,3,8,10,19,20,21,22,26,27,29,35,41,71,84,100,101,103,135])
print(data.shape)
dataLabels=np.array(data.columns)
print(dataLabels)
MENA=[]
MENA=['Algeria', 'Bahrain', 'Djibouti',' Egypt', 'Iran', 'Iraq', 'Jordan','Kuwait', 'Lebanon',' Libya', 'Malta',' Morocco', 'Oman',' Qatar', 'Saudi Arabia', 'Syria', 'Tunisia', 'United Arab Emirates', 'Palestine', 'Yemen','Ethiopia' ,' Sudan' ]

(181691, 19)
['iyear' 'imonth' 'iday' 'country_txt' 'region_txt' 'crit1' 'crit2'
 'crit3' 'doubtterr' 'success' 'suicide' 'attacktype1_txt' 'targtype1_txt'
 'natlty1_txt' 'claimed' 'weapsubtype1_txt' 'nkillter' 'nwound' 'nwoundte']


In [0]:
data_C=data.shape[1]
data_R=data.shape[0]
for c in range(data_C):
    Region_nums=[]
    Region_nums=data[dataLabels[c]].unique()
    Region_nums={Region_nums[i]: i for i in range(len(Region_nums))}
    Regions = Region_nums
    data[dataLabels[c]].replace(Regions, inplace=True)


In [168]:
data

,iyear,imonth,iday,country_txt,region_txt,crit1,crit2,crit3,doubtterr,success,suicide,attacktype1_txt,targtype1_txt,natlty1_txt,claimed,weapsubtype1_txt,nkillter,nwound,nwoundte
0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0.0,0,0.0,0.0,0.0
1,0,1,1,1,1,0,0,0,0.0,0,0,1,1,1,0.0,0,0.0,0.0,0.0
2,0,2,1,2,2,0,0,0,0.0,0,0,0,2,2,0.0,0,0.0,0.0,0.0
3,0,2,1,3,3,0,0,0,0.0,0,0,2,1,2,0.0,1,0.0,1.0,0.0
4,0,2,1,4,4,0,0,0,1.0,0,0,3,1,2,0.0,0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181686,46,12,22,81,7,0,0,1,2.0,0,0,4,5,84,2.0,2,1.0,3.0,1.0
181687,46,12,22,75,8,0,0,1,2.0,0,0,2,5,45,1.0,15,1.0,4.0,1.0
181688,46,12,22,2,2,0,0,0,0.0,0,0,3,0,22,1.0,7,1.0,0.0,1.0
181689,46,12,22,46,10,0,0,0,0.0,1,0,2,6,57,1.0,20,1.0,0.0,1.0


For computational reasons, the best subset cannot be applied for any large n due to the 2n complexity. Forward Stepwise begins with a model containing no predictors, and then adds predictors to the model, one at the time. At each step, the variable that gives the greatest additional improvement to the fit is added to the model.

In [0]:
def fit_linear_reg(X,Y):
    #Fit linear regression model and return RSS and R squared values
    model_k = linear_model.LinearRegression(fit_intercept = True)
    model_k.fit(X,Y)
    RSS = mean_squared_error(Y,model_k.predict(X)) * len(Y)
    R_squared = model_k.score(X,Y)
    return RSS, R_squared

In [0]:
#Importing tqdm for the progress bar
from tqdm import tnrange, tqdm_notebook
import itertools
from sklearn import linear_model
from sklearn.metrics import mean_squared_error

#Initialization variables
X =data
X = X[['iyear', 'imonth' ,'iday', 'country_txt' ,'region_txt' ,'crit1' ,'crit2',
 'crit3', 'doubtterr', 'success', 'suicide' ,'attacktype1_txt' ,'targtype1_txt',
 'natlty1_txt', 'claimed' ,'weapsubtype1_txt' ,'nkillter', 'nwound' ,'nwoundte']].dropna()
Y = X.pop('country_txt')
k = 10
remaining_features = list(X.columns.values)
features = []
RSS_list, R_squared_list = [np.inf], [np.inf] #Due to 1 indexing of the loop...
features_list = dict()

for i in range(1,k+1):
    best_RSS = np.inf
    
    for combo in itertools.combinations(remaining_features,1):

            RSS = fit_linear_reg(X[list(combo) + features],Y)   #Store temp result 

            if RSS[0] < best_RSS:
                best_RSS = RSS[0]
                best_R_squared = RSS[1] 
                best_feature = combo[0]

    #Updating variables for next loop
    features.append(best_feature)
    remaining_features.remove(best_feature)
    
    #Saving values for plotting
    RSS_list.append(best_RSS)
    R_squared_list.append(best_R_squared)
    features_list[i] = features.copy()

In [154]:
print('Forward stepwise subset selection')
print('Number of features |', 'Features |', 'RSS')
df1 = pd.concat([pd.DataFrame({'features':features_list}),pd.DataFrame({'RSS':RSS_list, 'R_squared': R_squared_list})], axis=1, join='inner')
df1['numb_features'] = df1.index

Forward stepwise subset selection
Number of features | Features | RSS


In [155]:
#Initializing useful variables
m = len(Y)
p = 5
hat_sigma_squared = (1/(m - p -1)) * min(df1['RSS'])

#Computing
df1['C_p'] = (1/m) * (df1['RSS'] + 2 * df1['numb_features'] * hat_sigma_squared )
df1['AIC'] = (1/(m*hat_sigma_squared)) * (df1['RSS'] + 2 * df1['numb_features'] * hat_sigma_squared )
df1['BIC'] = (1/(m*hat_sigma_squared)) * (df1['RSS'] +  np.log(m) * df1['numb_features'] * hat_sigma_squared )
df1['R_squared_adj'] = 1 - ( (1 - df1['R_squared'])*(m-1)/(m-df1['numb_features'] -1))
df1

,features,RSS,R_squared,numb_features,C_p,AIC,BIC,R_squared_adj
1,[natlty1_txt],2.235954e+08,0.213211,1,1230.648169,1.068833,1.068888,0.213206
2,"[natlty1_txt, iyear]",2.121451e+08,0.253502,2,1167.640403,1.014110,1.014221,0.253494
3,"[natlty1_txt, iyear, region_txt]",2.111910e+08,0.256859,3,1162.401607,1.009560,1.009727,0.256847
4,"[natlty1_txt, iyear, region_txt, suicide]",2.107335e+08,0.258469,4,1159.896427,1.007384,1.007606,0.258453
5,"[natlty1_txt, iyear, region_txt, suicide, clai...",2.102549e+08,0.260153,5,1157.274581,1.005107,1.005385,0.260133
6,"[natlty1_txt, iyear, region_txt, suicide, clai...",2.099970e+08,0.261061,6,1155.868080,1.003885,1.004219,0.261036
7,"[natlty1_txt, iyear, region_txt, suicide, clai...",2.097664e+08,0.261872,7,1154.611595,1.002794,1.003183,0.261844
8,"[natlty1_txt, iyear, region_txt, suicide, clai...",2.095562e+08,0.262612,8,1153.467161,1.001800,1.002245,0.262580
9,"[natlty1_txt, iyear, region_txt, suicide, clai...",2.093417e+08,0.263367,9,1152.299378,1.000786,1.001287,0.263330
10,"[natlty1_txt, iyear, region_txt, suicide, clai...",2.091911e+08,0.263896,10,1151.483430,1.000077,1.000633,0.263856


In [175]:
data2015.columns

Index(['Country', 'Region', 'Happiness Rank', 'Happiness Score',
       'Standard Error', 'Economy (GDP per Capita)', 'Family',
       'Health (Life Expectancy)', 'Freedom', 'Trust (Government Corruption)',
       'Generosity', 'Dystopia Residual'],
      dtype='object')

In [179]:
Names=df1.features[10]
dfData=data[Names]
newC=data['country_txt']
dfData['MENA'] = newC
newH=data2015['Happiness Score']
dfData['Happy'] = newH
newF=data2015['Freedom']
dfData['Freedom Score'] = newF
dfData[dfData['MENA'].isin(MENA)]
dfData


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http

,natlty1_txt,iyear,region_txt,suicide,claimed,weapsubtype1_txt,nwoundte,doubtterr,nwound,crit1,MENA,Happy,Freedom Score
0,Dominican Republic,1970,Central America & Caribbean,0,NaN,NaN,NaN,0.0,0.0,1,Dominican Republic,7.587,0.66557
1,Belgium,1970,North America,0,NaN,NaN,NaN,0.0,0.0,1,Mexico,7.561,0.62877
2,United States,1970,Southeast Asia,0,NaN,NaN,NaN,0.0,0.0,1,Philippines,7.527,0.64938
3,United States,1970,Western Europe,0,NaN,Unknown Explosive Type,NaN,0.0,NaN,1,Greece,7.522,0.66973
4,United States,1970,East Asia,0,NaN,NaN,NaN,-9.0,NaN,1,Japan,7.427,0.63297
...,...,...,...,...,...,...,...,...,...,...,...,...,...
181686,Somalia,2017,Sub-Saharan Africa,0,1.0,Unknown Gun Type,0.0,1.0,2.0,1,Somalia,NaN,NaN
181687,Russia,2017,Middle East & North Africa,0,0.0,"Projectile (rockets, mortars, RPGs, etc.)",0.0,1.0,7.0,1,Syria,NaN,NaN
181688,Philippines,2017,Southeast Asia,0,0.0,Arson/Fire,0.0,0.0,0.0,1,Philippines,NaN,NaN
181689,India,2017,South Asia,0,0.0,Grenade,0.0,0.0,0.0,1,India,NaN,NaN
